In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image

In [2]:
# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

In [3]:
# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255  # Normalize pixel values to [0, 1]
x_test = x_test.astype('float32') / 255    # Normalize pixel values to [0, 1]

y_train = to_categorical(y_train, 10)  # One-hot encode the labels
y_test = to_categorical(y_test, 10)    # One-hot encode the labels

In [4]:
# Build the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))  # 10 classes for CIFAR-10

D:\D2I\Codes\patra-toolkit\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.1)


Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 24s 23ms/step - accuracy: 0.3132 - loss: 1.8537 - val_accuracy: 0.5242 - val_loss: 1.3250
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.5320 - loss: 1.2989 - val_accuracy: 0.5858 - val_loss: 1.1873
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.6102 - loss: 1.1101 - val_accuracy: 0.6316 - val_loss: 1.0700
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.6561 - loss: 0.9903 - val_accuracy: 0.6678 - val_loss: 0.9691
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 17s 23ms/step - accuracy: 0.6886 - loss: 0.8940 - val_accuracy: 0.6750 - val_loss: 0.9468
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.7120 - loss: 0.8335 - val_accuracy: 0.7116 - val_loss: 0.8548
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 16s 23ms/step - accuracy: 0.7376 - loss: 0.7535 - val_accuracy: 0.6948 - val_loss: 0.9086
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.7546 - loss: 0.7099 - 

In [6]:
# Function to load and preprocess a local image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(32, 32))  # Resize to match CIFAR-10 input size
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dims to create a batch
    img_array = img_array.astype('float32') / 255  # Normalize
    return img_array

# Function to predict the class of a local image
def predict_image_class(img_path):
    img_array = load_and_preprocess_image(img_path)
    preds = model.predict(img_array)  # Make prediction
    return preds

In [7]:
# Replace 'your_image_path_here.jpg' with the path to your local image
image_path = r'D:\D2I\Codes\fork\patra-toolkit\examples\notebooks\data\images\kitty-cat-kitten-pet-45201.jpeg'  # Update this to your local image path
predictions = predict_image_class(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 804ms/step


In [8]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy:.4f}')
print(f'Test loss: {test_loss:.4f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7037 - loss: 0.8850
Test accuracy: 0.7053
Test loss: 0.8761


In [9]:
# Get the predicted class
predicted_class_idx  = np.argmax(predictions, axis=1)[0]
predicted_class_name = class_names[predicted_class_idx]

In [10]:
print(f'Predicted class index: {predicted_class_idx}')
print(f'Predicted class name: {predicted_class_name}')

Predicted class index: 3
Predicted class name: cat


In [11]:
from patra_toolkit import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis

D:\D2I\Codes\patra-toolkit\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
mc = ModelCard(
            name="CIFAR-10 Image Classification model",
            version="0.1",
            short_description="CNN model trained on CIFAR-10 dataset for image classification.",
            full_description="We have trained a Convolutional Neural Network (CNN) model using TensorFlow to classify images from the CIFAR-10 dataset. The dataset consists of 10 classes of images, including airplane, automobile, bird, cat, deer, dog, frog, horse, ship,and truck.",
            keywords="cifar-10, tensorflow, cnn, image classification, deep learning",
            input_type="Image",
            category="classification",
            foundational_model="None"
        )

mc.input_data = 'https://www.cs.toronto.edu/~kriz/cifar.html'
mc.output_data = 'https://github.com/yourusername/cifar10_cnn_model'

In [13]:
ai_model = AIModel(
            name="CIFAR-10 Image Classification CNN model",
            version="0.1",
            description="Convolutional Neural Network (CNN) for classifying images from the CIFAR-10 dataset.",
            owner="Isuru Gamage",
            location="https://github.com/yourusername/cifar10_cnn_model",
            license="BSD-3 Clause",
            framework="tensorflow",
            model_type="cnn",
            test_accuracy=test_accuracy
        )
ai_model.populate_model_structure(model)

In [14]:
ai_model.add_metric("Test loss", test_loss)
ai_model.add_metric("Epochs", 10)
ai_model.add_metric("Batch Size", 64)
ai_model.add_metric("Optimizer", "Adam")
ai_model.add_metric("Learning Rate", 0.0001)

In [15]:
mc.ai_model = ai_model

In [16]:
mc.populate_requirements()

In [17]:
print(mc)

{
    "name": "CIFAR-10 Image Classification model",
    "version": "0.1",
    "short_description": "CNN model trained on CIFAR-10 dataset for image classification.",
    "full_description": "We have trained a Convolutional Neural Network (CNN) model using TensorFlow to classify images from the CIFAR-10 dataset. The dataset consists of 10 classes of images, including airplane, automobile, bird, cat, deer, dog, frog, horse, ship,and truck.",
    "keywords": "cifar-10, tensorflow, cnn, image classification, deep learning",
    "author": "Isuru Gamage",
    "input_type": "Image",
    "category": "classification",
    "input_data": "https://www.cs.toronto.edu/~kriz/cifar.html",
    "output_data": "https://github.com/yourusername/cifar10_cnn_model",
    "ai_model": {
        "name": "CIFAR-10 Image Classification CNN model",
        "version": "0.1",
        "description": "Convolutional Neural Network (CNN) for classifying images from the CIFAR-10 dataset.",
        "owner": "Isuru Gamage"

In [18]:
mc.validate()

True

In [ ]:
mc.submit("http://127.0.0.1:5002")

In [ ]:
mc.save("/Users/swithana/git/d2i/patra-toolkit/examples/model_cards/CNN_Example_MC.json")